**predicting Drive**

In [ ]:
# Importing important libraries
import pyforest
from lazypredict.Supervised import LazyRegressor
from pandas.plotting import scatter_matrix
# Scikit-learn packages
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import metrics
from sklearn.metrics import mean_squared_error
# Hide warnings
import warnings
warnings.filterwarnings('ignore')
# Setting up max columns displayed to 100
pd.options.display.max_columns = 100

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


<IPython.core.display.Javascript object>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
sns.set()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Reading the file

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/capstone_2_final_preprocessed_data.csv')

In [ ]:
data=data.drop('Unnamed: 0',axis=1)

In [ ]:
data.head()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,yrdln,yrdline100,ydstogo,ydsnet,GoalToGo,FirstDown,posteam,DefensiveTeam,desc,PlayAttempted,Yards.Gained,sp,Touchdown,Safety,Onsidekick,PlayType,Passer_ID,PassAttempt,PassOutcome,PassLength,AirYards,YardsAfterCatch,QBHit,PassLocation,InterceptionThrown,Rusher_ID,RushAttempt,RunLocation,RunGap,Receiver_ID,Reception,Fumble,RecFumbTeam,RecFumbPlayer,Sack,Challenge.Replay,Accepted.Penalty,Penalty.Yards,PosTeamScore,DefTeamScore,ScoreDiff,AbsScoreDiff,HomeTeam,AwayTeam,Timeout_Indicator,Timeout_Team,posteam_timeouts_pre,HomeTimeouts_Remaining_Pre,AwayTimeouts_Remaining_Pre,HomeTimeouts_Remaining_Post,AwayTimeouts_Remaining_Post,No_Score_Prob,Opp_Field_Goal_Prob,Opp_Safety_Prob,Opp_Touchdown_Prob,Field_Goal_Prob,Safety_Prob,Touchdown_Prob,ExPoint_Prob,TwoPoint_Prob,ExpPts,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2009-09-10,2009091000,14,3,2.00,12:06,13,1626.00,54,PIT,42.00,42.00,10,38,0.00,1.00,TEN,PIT,(12:06) K.Collins pass short left to B.Scaife ...,1,1,0,0,0,0,Pass,00-0003292,1,Complete,Short,1,0,0,left,0,None,0,left,tackle,00-0023612,1,1,PIT,K.Fox,0,0,0,0,7.00,7.00,0.00,-0.00,PIT,TEN,0,None,3,3,3,3,3,0.00,0.08,0.00,0.13,0.36,0.00,0.42,0.00,0.00,2.92,-5.61,-5.61,0.00,0.42,0.58,0.57,0.43,0.58,-0.16,-0.16,0.00,2009
1,2009-09-10,2009091000,15,3,4.00,10:12,11,1512.00,48,TEN,45.00,45.00,1,9,0.00,1.00,PIT,TEN,(10:12) (Punt formation) D.Sepulveda punts 38 ...,1,0,0,0,0,0,Punt,None,0,Complete,Short,0,0,0,left,0,None,0,left,tackle,None,0,1,PIT,K.Fox,0,0,0,0,7.00,7.00,0.00,-0.00,PIT,TEN,0,None,3,3,3,3,3,0.00,0.15,0.00,0.24,0.30,0.01,0.31,0.00,0.00,0.96,0.10,0.00,0.00,0.52,0.48,0.55,0.45,0.52,0.03,0.00,0.00,2009
2,2009-09-10,2009091000,23,4,1.00,01:03,2,63.00,57,TEN,34.00,34.00,10,54,0.00,1.00,PIT,TEN,(1:03) (Shotgun) B.Roethlisberger pass deep mi...,1,30,0,0,0,0,Pass,00-0022924,1,Complete,Deep,16,14,0,middle,0,None,0,left,tackle,00-0017162,1,1,TEN,S.Tulloch,0,0,0,0,10.00,10.00,0.00,-0.00,PIT,TEN,0,None,2,2,3,2,3,0.38,0.02,0.00,0.02,0.31,0.00,0.27,0.00,0.00,2.59,-2.77,0.66,-3.43,0.80,0.20,0.42,0.58,0.80,-0.38,0.06,-0.44,2009
3,2009-09-13,2009091304,7,2,0.00,11:09,12,2469.00,51,MIN,30.00,30.00,0,0,0.00,1.00,CLE,MIN,R.Longwell kicks 70 yards from MIN 30 to CLE 0...,1,27,0,0,0,0,Kickoff,None,0,Complete,Deep,0,0,0,middle,0,None,0,left,tackle,None,0,1,TEN,S.Tulloch,0,0,0,0,3.00,10.00,-7.00,7.00,CLE,MIN,0,None,3,3,3,3,3,0.13,0.15,0.01,0.23,0.19,0.00,0.29,0.00,0.00,0.59,0.43,0.00,0.00,0.27,0.73,0.28,0.72,0.27,0.01,0.00,0.00,2009
4,2009-09-13,2009091304,7,2,2.00,10:17,11,2417.00,43,CLE,27.00,73.00,8,73,0.00,1.00,CLE,MIN,(10:17) B.Quinn pass short middle to L.Vickers...,1,12,0,0,0,0,Pass,00-0025409,1,Complete,Short,2,10,0,middle,0,None,0,left,tackle,00-0024394,1,1,TEN,S.Tulloch,0,1,0,0,3.00,10.00,-7.00,7.00,CLE,MIN,1,MIN,3,3,3,3,2,0.14,0.15,0.00,0.22,0.20,0.00,0.29,0.00,0.00,0.62,1.51,-0.52,2.03,0.27,0.73,0.34,0.66,0.27,0.07,-0.02,0.08,2009


This dataset has columns representing various types of probabilities. Since those probability equations are not part of the raw data, I am dropping them.

In [ ]:
data.drop(['No_Score_Prob', 'Opp_Field_Goal_Prob', 'Opp_Safety_Prob', 'Opp_Touchdown_Prob', 
         'Field_Goal_Prob','Safety_Prob', 'Touchdown_Prob', 'ExPoint_Prob', 'TwoPoint_Prob', 'ExpPts', 
         'EPA', 'airEPA', 'yacEPA','Home_WP_pre', 'Away_WP_pre', 'Home_WP_post', 'Away_WP_post', 
         'Win_Prob', 'WPA', 'airWPA','yacWPA'],axis=1,inplace=True)

There is also a number of columns that have no significance in Drive. These columns will also be dropped.

In [ ]:
data.drop(['GameID', 'time', 'Date', 'ydsnet','desc', 'Passer_ID',
           'Rusher_ID', 'Receiver_ID','Challenge.Replay','Penalty.Yards','Timeout_Team', 
         'posteam_timeouts_pre', 'HomeTimeouts_Remaining_Pre', 'AwayTimeouts_Remaining_Pre',
         'HomeTimeouts_Remaining_Post', 'AwayTimeouts_Remaining_Post', 'sp'], axis=1, 
          inplace=True)

In [ ]:
data.shape

(362447, 45)

Using label encoder on categorical features

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data['SideofField']=le.fit_transform(data['SideofField'])
data['posteam']=le.fit_transform(data['posteam'])
data['DefensiveTeam']=le.fit_transform(data['DefensiveTeam'])
data['PlayType']=le.fit_transform(data['PlayType'])
data['PassOutcome']=le.fit_transform(data['PassOutcome'])
data['PassLength']=le.fit_transform(data['PassLength'])
data['PassLocation']=le.fit_transform(data['PassLocation'])
data['RunLocation']=le.fit_transform(data['RunLocation'])
data['RunGap']=le.fit_transform(data['RunGap'])
data['RecFumbTeam']=le.fit_transform(data['RecFumbTeam'])
data['RecFumbPlayer']=le.fit_transform(data['RecFumbPlayer'])
data['HomeTeam']=le.fit_transform(data['HomeTeam'])
data['AwayTeam']=le.fit_transform(data['AwayTeam'])
data['Season']=le.fit_transform(data['Season'])


In [ ]:

y = data['Drive']

X = data.drop('Drive', axis=1)

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 123, stratify=y)

<IPython.core.display.Javascript object>

In [ ]:
pip install mlrose

Usming SMOTE to balance the target variable

In [ ]:
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose
from imblearn.over_sampling import SMOTE

X_train_new, y_train_new = SMOTE().fit_sample(X_train, y_train)

print(pd.Series(y_train_new).value_counts())

35    12805
9     12805
15    12805
14    12805
13    12805
12    12805
11    12805
10    12805
8     12805
17    12805
7     12805
6     12805
5     12805
4     12805
3     12805
2     12805
16    12805
18    12805
34    12805
27    12805
33    12805
32    12805
31    12805
30    12805
29    12805
28    12805
26    12805
19    12805
25    12805
24    12805
23    12805
22    12805
21    12805
20    12805
1     12805
dtype: int64


In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(X_train,y_train)
print(model.score(X_train,y_train))
print(model.score(X_test,y_test))

0.9934793035659716
0.953558591391863
